# Projeto 1 - Ciência dos Dados

Nome: Guilermo Kuznietz

Nome: Jonas Bonfá Pelegrina

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/jonas/OneDrive - Insper - Institudo de Ensino e Pesquisa/2021.2/Ciência dos Dados/P1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'alexa_b.xlsx'

In [4]:
train = pd.read_excel(filename)
train.tail()

,Treinamento,Classificacao
295,la música da vida,0
296,alexa play me odio by matias candia,0
297,cemre:vocês queriam tanto que eu aparecesse qu...,0
298,@karnecita alexa: https://t.co/tc5hbjcuuy,0
299,"alexa, da mis clases híbridas.",0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificacao
0,eu odeio quando eu peço pra alexa tocar a musi...,1
1,alexa play digital dash,1
2,muito bom conversar com a alexa,1
3,perguntei pra alexa se podia chamar ela de jar...,1
4,e a alexa que tá fazendo elogios antes de dar ...,1


___
## Classificador automático de sentimento


<img src="img/alexa_logo.png" alt="Alexa logo" width="500" height="600">

Nosso projeto busca entender o uso da Alexa no mercado brasileiro como os usuário interagem com ela e quais são as principais reações sobre o produto.

Exemplos de casos relevantes:
- Alexa,como está o dia hoje?
- Eu e minha família adoramos a Alexa
- A Alexa me entende como vivia antes sem ela?

Exemplos de casos irrelevante:
- Queria comprar uma Alexa
- Eu amo minha amiga que chama Alexa
- Tenho quatro filhas: Beatriz,Maria e Alexa

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13

In [7]:
#Limpeza de mensagens removendo os caracteres especiais
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    

In [8]:
cleanup('@duda_barroso1 alexa, o que tenho pra hj?')

'duda_barroso1 alexa o que tenho pra hj'

In [9]:
#RELEVANTE
train_relevante = train.loc[train['Classificacao'] == 1,'Treinamento']
lista = []
lista_relevante = []

for i in range(0,len(train_relevante)):
    lista.append(cleanup(train_relevante[i]).split())

for lista1 in lista:
    for palavra in lista1:
        if "https" not in palavra:
            lista_relevante.append(palavra)

#RELEVANTE
train_irrelevante = train.loc[train['Classificacao'] == 0,'Treinamento']
lista = []
lista_irrelevante = []

for frase in train_irrelevante:
    lista.append(cleanup(frase).split())

for lista1 in lista:
    for palavra in lista1:
        if "https" not in palavra:
            lista_irrelevante.append(palavra)

**Verificando as frequências**

In [25]:
# RELEVANTE - FREQUENCIA ABSOLUTA
serie_relevante = pd.Series(lista_relevante)
serie_relevante.value_counts()

alexa        156
a             89
eu            59
e             46
pra           45
            ... 
aipo           1
menos          1
programar      1
scott          1
prof           1
Length: 796, dtype: int64

In [24]:
# RELEVANTE - FREQUENCIA RELATIVA
serie_relevante = pd.Series(lista_relevante)
serie_relevante_relativa = serie_relevante.value_counts(True)

In [23]:
# IRRELEVANTE - FREQUENCIA ABSOLUTA
serie_irrelevante = pd.Series(lista_irrelevante)
serie_irrelevante.value_counts()

alexa             86
de                47
a                 41
e                 36
que               30
                  ..
era                1
marrycartl29hk     1
noloveforme404     1
50nano75           1
apareço            1
Length: 951, dtype: int64

In [22]:
# IRRELEVANTE - FREQUENCIA RELATIVA
serie_irrelevante = pd.Series(lista_irrelevante)
serie_irrelevante_relativa = serie_irrelevante.value_counts(True)

In [39]:
# TRANSFORMANDO EM UM GRANDE TEXTO

livro_relevante = " ".join(lista_relevante)

livro_irrelevante = " ".join(lista_irrelevante)

In [48]:
# JUNTANDO TODAS AS PALAVRAS
todas_palavras = livro_relevante + livro_irrelevante
todas_palavras = todas_palavras.split()
series_todas_palavras = pd.Series(todas_palavras)
series_todas_palavras_relativas = series_todas_palavras.value_counts(True)
print(series_todas_palavras_relativas)

alexa          0.060553
a              0.032663
de             0.022613
e              0.020603
eu             0.019849
                 ...   
utilidade      0.000251
menos          0.000251
aipo           0.000251
tradicional    0.000251
prof           0.000251
Length: 1535, dtype: float64


**PROBABILIDADES**

In [26]:
serie_relevante_relativa['alexa']

0.07482014388489208

**P(R) E P(I)**

In [49]:
probR = len(serie_relevante) / len(series_todas_palavras)
probR

0.5238693467336684

In [52]:
probI = len(serie_irrelevante) / len(series_todas_palavras)
probI

0.4763819095477387

**Começando a calcular**

In [67]:
#Como pegar as frases,pois essa está com a vírgula e as da lista estão todas juntas
frase_quebrada = train_relevante[0].split()
frase_quebrada_series = pd.Series(frase_quebrada)
frase_quebrada_series[3]

'pra'

In [69]:
a = serie_relevante_relativa[frase_quebrada_series].prod()

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index([\'"toca\', \'gostosa",\'], dtype=\'object\'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**